In [2]:
#Link to the environment
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Instalar las bibliotecas necesarias
!pip install gradio joblib spotipy

# Configurar la autenticación de Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '28c78bd5b1fb4e698d23a84bc44c2379'
client_secret = '791c1db330134c4691660181e9a656bf'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB

In [ ]:
# Cargar el modelo entrenado y el PCA
import joblib

# Cargar los modelos guardados desde Google Drive
svm_model = joblib.load('/content/drive/MyDrive/ProyectoIA/TerceraEntrega/models/svm.pkl')
decision_tree_model = joblib.load('/content/drive/MyDrive/ProyectoIA/TerceraEntrega/models/decision_tree.pkl')
logistic_reg_model = joblib.load('/content/drive/MyDrive/ProyectoIA/TerceraEntrega/models/logistic_reg.pkl')
# Cargar el PCA
pca_6 = joblib.load('/content/drive/MyDrive/ProyectoIA/TerceraEntrega/models/pca.pkl')

# Definir la función de obtención de métricas de canciones
import numpy as np

def get_song_metrics(song_name):
    try:
        print(f"Searching for song: {song_name}")
        results = sp.search(q=song_name, limit=1, type='track')
        if len(results['tracks']['items']) == 0:
            print("Song not found in Spotify database.")
            return None  # Si no se encuentra la canción, devolver None

        track = results['tracks']['items'][0]
        track_id = track['id']
        print(f"Found track ID: {track_id}")

        features = sp.audio_features(track_id)[0]
        print(f"Audio features: {features}")

        # Seleccionar solo las métricas necesarias
        metrics = np.array([[features['acousticness'], features['danceability'], features['energy'],
                             features['instrumentalness'], features['liveness'], features['speechiness'],
                             features['valence'], features['tempo'], features['duration_ms']]])
        print(f"Metrics for PCA: {metrics}")
        return metrics
    except Exception as e:
        print(f"Error obtaining song metrics: {e}")
        return None

# Definir la función de predicción que utilice todos los modelos
def predict_genre(song_name):
    features = get_song_metrics(song_name)
    if features is None:
        return "Song not found or metrics not available."

    try:
        # Aplicar PCA a las características de entrada
        features_pca = pca_6.transform(features)
        print(f"PCA transformed features: {features_pca}")

        # Realizar las predicciones con todos los modelos
        prediction_svm = svm_model.predict(features_pca)
        prediction_dt = decision_tree_model.predict(features_pca)
        prediction_lr = logistic_reg_model.predict(features_pca)

        genre_svm = 'Rock' if prediction_svm[0] == 1 else 'Hip-Hop'
        genre_dt = 'Rock' if prediction_dt[0] == 1 else 'Hip-Hop'
        genre_lr = 'Rock' if prediction_lr[0] == 1 else 'Hip-Hop'

        result = (
            f"SVM: Genre: {genre_svm}\n"
            f"Decision Tree: Genre: {genre_dt}\n"
            f"Logistic Regression: Genre: {genre_lr}"
        )

        return result
    except Exception as e:
        print(f"Error in prediction: {e}")
        return f"An error occurred during prediction: {e}"

# Crear la interfaz de Gradio
import gradio as gr

interface = gr.Interface(
    fn=predict_genre,
    inputs=gr.Textbox(label="Song Name"),
    outputs=gr.Textbox(label="Genre Prediction"),
    title="Music Genre Prediction",
    description="Enter the name of a song to predict its genre (Rock or Hip-Hop) based on its audio features."
)

interface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2f3d70899ccf548fd9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Searching for song: Father's Day
Found track ID: 166v02Sp99FfoyM4iurgoj
Audio features: {'danceability': 0.96, 'energy': 0.387, 'key': 0, 'loudness': -9.751, 'mode': 1, 'speechiness': 0.0921, 'acousticness': 0.882, 'instrumentalness': 0, 'liveness': 0.196, 'valence': 0.968, 'tempo': 116.089, 'type': 'audio_features', 'id': '166v02Sp99FfoyM4iurgoj', 'uri': 'spotify:track:166v02Sp99FfoyM4iurgoj', 'track_href': 'https://api.spotify.com/v1/tracks/166v02Sp99FfoyM4iurgoj', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/166v02Sp99FfoyM4iurgoj', 'duration_ms': 184320, 'time_signature': 4}
Metrics for PCA: [[8.82000e-01 9.60000e-01 3.87000e-01 0.00000e+00 1.96000e-01 9.21000e-02
  9.68000e-01 1.16089e+02 1.84320e+05]]
PCA transformed features: [[ 80511.44921889  -3819.35581254  49691.92091794 -86388.27493307
   76937.27753251 -32291.38003048]]
Searching for song: Hundred-Year Flood
Found track ID: 6t3REO1y1yxQp1Rb3J1v8P
Audio features: {'danceability': 0.223, 'energy': 0.767, 'key':